In [ ]:
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import statsmodels.api as sm
import pickle

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
# Load results file
def extract_data_from_obj(obj):
    samples = []
    labels = []
    pz_decay = []
    item = []
    pz_empowered_mu = []
    for data in obj:
        samples.append(data['sampled_s'])
        labels.append(data['label'])
        pz_decay.append(data['pz_decay'])
        item.append(data['skill'])
        pz_empowered_mu.append(data['pz_empowered_mu'])
    labels = torch.cat(labels,0)
    pz_decay = torch.cat(pz_decay, 0)
    items = torch.cat(item, 0)
    sampled_s = torch.cat(samples, 1)
    
    return labels, pz_decay, items, sampled_s

object_file = '../groupkt.obj'

# Behavioural data
file = open(object_file,'rb')
obj = pickle.load(file)
file.close()
perf, decay, items, s_sample = extract_data_from_obj(obj)
items = items.detach().cpu().numpy()
perf = perf.detach().cpu().numpy()

# Graph data
model_path = '../groupkt.pt'
groupkt = torch.load(model_path)
node_emb = groupkt['node_dist.node_u']
node_trans = groupkt['node_dist.transformation_layer']

# p of direction
w_direction = node_emb @ (node_trans - node_trans.transpose(-1,-2)) @ node_emb.transpose(-1,-2)
p_direction = torch.sigmoid(w_direction).cpu().numpy()

# p of existing
num_node = 263 # Assist12
w_similarity = node_emb @ node_emb.transpose(-1,-2)
w_similarity = w_similarity.cpu() * (1-torch.eye(num_node))
p_sim = torch.sigmoid(w_similarity).numpy()
sim_origin = w_similarity.numpy()

# p of both
p_edge = p_sim * p_direction

In [ ]:
# Extract transition matrix
num_node = 263 # Assist12
gap = 1
T = np.zeros((num_node, num_node, 4)) # 0-1, 0-0, 1-1, 1-0
N = np.zeros((num_node, num_node))

for l in range(items.shape[0]):
    correct = perf[l]
    index = items[l]
    
    for i in range(10-gap):
        if index[i+gap] != index[i]:
            if correct[i] == 0:
                if correct[i+gap] == 1:
                    T[index[i], index[i+gap], 0]+=1
                else:
                    T[index[i], index[i+gap], 1]+=1
            else:
                if correct[i+gap] == 1:
                    T[index[i], index[i+gap], 2]+=1
                else:
                    T[index[i], index[i+gap], 3]+=1
            N[index[i], index[i+gap]] += 1
            
print('maximum transition times: ', N.max())
print('amount of different transition: ', (N!=0).sum())
Nc_minus = T[..., 0] + T[..., 1]
Nc_plus = T[..., 2] + T[..., 3]
Ne_minus = T[..., 1] + T[..., 3]
Ne_plus = T[..., 0] + T[..., 2]

In [ ]:
num_sample = 10000

# Compute likelihood based on behavioural data
# P(D|G0)
w0 = np.arange(0,num_sample,1)/num_sample
w0 = w0.reshape(num_sample, 1, 1).repeat(num_node, 1).repeat(num_node, -1)
p0 = np.power(w0, np.expand_dims(Ne_plus, 0).repeat(num_sample, 0)) * np.power(1-w0, np.expand_dims(Ne_minus, 0).repeat(num_sample, 0))

# P (D|G1)
w0 = np.arange(0,num_sample,1)/num_sample
w0 = w0.reshape(num_sample, 1, 1).repeat(num_node, 1).repeat(num_node, -1)
w0 = w0.repeat(num_sample, 0)
w1 = np.arange(0,num_sample,1)/num_sample 
w1 = w1.reshape(num_sample, 1, 1).repeat(num_node, 1).repeat(num_node, -1)
w1 = np.tile(w1, (num_sample, 1, 1))

N_e1_c1 = np.expand_dims(T[..., 2], 0).repeat(num_sample*num_sample, 0)
p_e1_c1 = np.power(np.multiply(w0, 1-w1), N_e1_c1)

N_e1_c0 = np.expand_dims(T[..., 0], 0).repeat(num_sample*num_sample, 0)
p_e1_c0 = np.power(w0, N_e1_c0)

p1 = np.multiply(p_e1_c1, p_e1_c0)

# causal support
support = np.log(p1.mean(0) + 1e-6) - np.log(p0.mean(0) + 1e-6)

In [ ]:
# Regression
mask = (T[...,2] + T[..., 3] + T[...,0] + T[..., 1] >1)

data = {'performance': support[mask].flatten(),
        'mu': p_edge[mask].flatten()
       }
data = pd.DataFrame(data)
data = sm.add_constant(data)

y = data['performance']
X = data[['const', 'mu']]

model = sm.OLS(y, X)
result = model.fit()

print(result.summary())